背景：



In [3]:
import os
import sys

current_dir = os.path.dirname(os.path.realpath('__file__'))
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
sys.path.append(parent_dir)

from apikey import apikey, langchain_apikey

os.environ['OPENAI_API_KEY'] = apikey
os.environ['LANGCHAIN_API_KEY'] = langchain_apikey
os.environ["LANGCHAIN_TRACING_V2"] = "true"


# 1.记录到特定项目

如“概念”部分所述，LangSmith 使用项目的概念对Trace进行分组。

如果未指定，则跟踪器项目将设置为default。您可以设置 LANGCHAIN_PROJECT 环境变量，以便为整个应用程序运行配置自定义项目名称。这应该在执行程序之前完成。

In [ ]:
export LANGCHAIN_PROJECT="My Project"



# 2.在运行时更改目标项目

当全局环境变量过于宽泛时，还可以在程序运行时设置项目名称。当您想要将跟踪记录到同一应用程序中的不同项目时，这很有用。

In [13]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant."),
    ("user", "Question: {question}")
])

model = ChatOpenAI(model="gpt-3.5-turbo")
output_parser = StrOutputParser()

chain = prompt | model | output_parser



## 2-1. 直接通过langchain

In [15]:
# You can set the project name for a specific tracer instance:
from langchain.callbacks.tracers import LangChainTracer

tracer = LangChainTracer(project_name="My Project")

question = "How many people live in canada as of 2023?"

chain.invoke({"question": question}, config={"callbacks": [tracer]})




'As of 2023, the estimated population of Canada is around 38 million people.'

In [17]:

# LangChain python also supports a context manager for tracing a specific block of code.
# You can set the project name using the project_name parameter.
from langchain_core.tracers.context import tracing_v2_enabled

with tracing_v2_enabled(project_name="My Project"):
    chain.invoke({"question": "How many people live in china as of 2023?"})


## 2-2. 通过python SDK


In [18]:
import openai
from langsmith import traceable
from langsmith.run_trees import RunTree

client = openai.Client()

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
]

@traceable(
    run_type="llm",
    name="OpenAI Call Decorator",
)
def call_openai(
    messages: list[dict], model: str = "gpt-3.5-turbo"
) -> str:
    return client.chat.completions.create(
        model=model,
        messages=messages,
    ).choices[0].message.content

# You can specify the Project via the project_name parameter
call_openai(
    messages,
    langsmith_extra={"project_name": "My Project"},
)


'Hello! How can I help you today?'

# 3.把metadata 和 tags 加入到traces中


## 3-1. 通过langchain

In [19]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [24]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI."),
    ("user", "{input}")
])

chat_model = ChatOpenAI()
output_parser = StrOutputParser()

chain = (prompt | chat_model | output_parser).with_config({"tags": ["top-level-tag"], 
                                                           "metadata": {"top-level-key": "top-level-value"}})


chain.invoke({"input": "What is the meaning of life?"}, {"tags": ["shared-tags"], "metadata": {"shared-key": "shared-value"}})




'The meaning of life is a complex and subjective concept that can vary greatly from person to person. Some may find meaning in personal relationships, fulfilling work, spiritual beliefs, or contributing to the greater good of society. Ultimately, the meaning of life is a deeply personal question that each individual must explore and determine for themselves.'

# 4.自定义Run name

# 5.掩盖输入和输出

In [26]:


os.environ["LANGCHAIN_HIDE_INPUTS"] = "true"
os.environ["LANGCHAIN_HIDE_OUTPUTS"] = "true"


In [27]:
from langchain_core.tracers.context import tracing_v2_enabled
from langchain_openai import ChatOpenAI
from langsmith import Client


def filter_inputs(inputs: dict):
    # You can define custom filtering here
    return {}


def filter_outputs(outputs: dict):
    # You can define custom filtering here
    return {}


llm = ChatOpenAI()



In [28]:
# You can configure tracing using the context manager below
# or by directly creating a LangChainTracer object
with tracing_v2_enabled(
    "test-filtering",
    client=Client(hide_inputs=filter_inputs, hide_outputs=filter_outputs),
) as cb:
    llm.invoke("Say foo")
    # The linked run will have its metadata present, but the inputs will be hidden
    print(cb.get_run_url())

with tracing_v2_enabled("test-filtering", client=Client()) as cb:
    llm.invoke("Say bar")
    # The linked run will not have hidden inputs and outputs
    print(cb.get_run_url())

https://smith.langchain.com/o/1040bdf6-7131-5c03-b7be-fb1b4779027f/projects/p/119422db-340f-4b6a-a8a1-303b804983a3/r/77f3af42-83da-488e-953c-60d3929cd251?poll=true
https://smith.langchain.com/o/1040bdf6-7131-5c03-b7be-fb1b4779027f/projects/p/119422db-340f-4b6a-a8a1-303b804983a3/r/002e13d4-08c2-4285-ae8f-4be58c3103e3?poll=true
